In [22]:
import pandas as pd

In [23]:
# List of CSV file paths
file_paths = [
    # 'Collected Dataset/fist.csv',
    # 'Collected Dataset/hand_open.csv',
    # 'Collected Dataset/index_middle_thumb.csv',
    'Collected Dataset/index_middle.csv',
    'Collected Dataset/index_middle2.csv',
    # 'Collected Dataset/index_thumb.csv',
    'Collected Dataset/index.csv',
    # 'Collected Dataset/pinch.csv',
    # 'Collected Dataset/peace.csv',
    'Collected Dataset/thumbs_pinky.csv',
    'Collected Dataset/thumbs_pinky2.csv',
    # # 'Collected Dataset/thumbs_down.csv',
    # # 'Collected Dataset/thumbs_up.csv',
    'Collected Dataset/right/fist.csv',
    'Collected Dataset/right/hand_open.csv',
    'Collected Dataset/right/index_middle_thumb.csv',
    'Collected Dataset/right/index_middle.csv',
    'Collected Dataset/right/index_thumb.csv',
    'Collected Dataset/right/index.csv',
    'Collected Dataset/right/pinch.csv',
    'Collected Dataset/right/peace.csv',
    'Collected Dataset/right/thumbs_pinky.csv',
    'Collected Dataset/right/thumbs_down.csv',
    'Collected Dataset/right/thumbs_up.csv',
]


In [24]:
csv_data = [pd.read_csv(file) for file in file_paths]

# Concatenate all DataFrames into a single DataFrame
data = pd.concat(csv_data, ignore_index=True)

# Shuffle the DataFrame
data = data.sample(frac=1).reset_index(drop=True)

In [25]:
# remove missing values
df = data.dropna()

print(df.shape)

(109597, 64)


In [26]:
string_to_numeric = {'index': 0, 'index_middle': 1, 'index_thumb': 2, 'index_middle_thumb' : 3, 'peace': 4, 'hand_open' : 5, 'fist': 6, 'pinch' : 7, 'thumbs_up' : 8, 'thumbs_down': 9 , 'thumbs_pinky' : 10}

X = df.iloc[:, :-1]
y = df.iloc[:, -1]

y.replace(string_to_numeric, inplace=True)

X.sample(1)

# remove column names
X = X.values

X.shape

C:\Users\ramez\AppData\Local\Temp\ipykernel_25732\4026723423.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y.replace(string_to_numeric, inplace=True)


(109597, 63)

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.3, random_state=12)

X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=12)

print("Training data shape:", X_train.shape)
print("Testing data shape:", X_test.shape)
print("Validation data shape:", X_val.shape)

Training data shape: (76717, 63)
Testing data shape: (16440, 63)
Validation data shape: (16440, 63)


In [60]:
# apply pca

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(n_components=0.99)

X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
X_val_pca = pca.transform(X_val)

print("Training data shape after PCA:", X_train_pca.shape)

Training data shape after PCA: (76717, 21)


In [61]:
# save pca model
import pickle

# with open('../utils/pca.pkl', 'wb') as f:
#     pickle.dump(pca, f)

In [30]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Hyperparameter tuned models

my_models = {
    ("KNN", KNeighborsClassifier(algorithm = 'auto', n_neighbors = 3)),
    ("Decision Tree", DecisionTreeClassifier(criterion = 'entropy', max_depth = 30, min_samples_split = 2, splitter = 'best')),
    ("Random Forest", RandomForestClassifier(criterion = 'entropy', max_depth = 20, n_estimators = 50)),
    ("XGBoost", XGBClassifier(learning_rate=0.2, max_depth=6, n_estimators= 50))
}

In [31]:
from sklearn.metrics import confusion_matrix
import time 

for name, model in my_models:
    print("Training", name)
    model.fit(X_train_pca, y_train)
    print("Training Accuracy:", model.score(X_train_pca, y_train))
    print("Validation Accuracy:", model.score(X_val_pca, y_val))
    print("Testing Accuracy:", model.score(X_test_pca, y_test))
    print()

Training XGBoost
Training Accuracy: 1.0
Validation Accuracy: 0.9998175182481752
Testing Accuracy: 0.9994525547445255

Training Random Forest
Training Accuracy: 1.0
Validation Accuracy: 0.9998175182481752
Testing Accuracy: 0.9996350364963503

Training KNN
Training Accuracy: 0.9999608952383435
Validation Accuracy: 0.9997566909975669
Testing Accuracy: 0.9996958637469586

Training Decision Tree
Training Accuracy: 1.0
Validation Accuracy: 0.9980535279805353
Testing Accuracy: 0.9985401459854014



In [32]:
for name, model in my_models:
    print("Model:", name)
    start = time.time()
    y_pred = model.predict(X_test_pca)
    end = time.time()
    print("Time taken for prediction:", end-start)
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    print()
    print("Testing Accuracy:", model.score(X_test_pca, y_test))
    print()
    print()

Model: XGBoost
Time taken for prediction: 0.05799722671508789
[[1953    2    0    0    0    0    0    0    0    0    0]
 [   1 2621    0    0    0    0    3    0    0    0    0]
 [   0    0 1388    0    0    0    0    0    0    0    0]
 [   0    0    0 1426    0    0    0    0    0    0    0]
 [   0    0    0    0 1443    0    0    0    0    0    0]
 [   0    0    0    0    0 1244    0    0    0    0    0]
 [   2    0    0    0    0    0  995    0    0    0    0]
 [   0    0    0    0    0    0    0 1233    0    0    0]
 [   0    0    0    0    0    0    0    0  862    0    0]
 [   0    0    0    0    0    0    0    0    0 1001    0]
 [   0    0    0    0    0    1    0    0    0    0 2265]]

Testing Accuracy: 0.9994525547445255


Model: Random Forest
Time taken for prediction: 0.10700225830078125
[[1955    0    0    0    0    0    0    0    0    0    0]
 [   3 2619    0    0    0    0    3    0    0    0    0]
 [   0    0 1388    0    0    0    0    0    0    0    0]
 [   0    0    0 

# Models

In [63]:
import tensorflow as tf
from tensorflow.keras import layers, models

num_states = len(y.unique())

print(y.unique())

tf_model = models.Sequential([
    layers.Input(shape=(X_train_pca.shape[1],)), 
    # tf.keras.layers.Dense(24, activation= 'relu'), 
    tf.keras.layers.Dense(64, activation= 'relu'), 
    # tf.keras.layers.Dense(128, activation= 'relu'), 
    tf.keras.layers.Dense(num_states, activation= 'softmax')
])
                                
# Compile the model
tf_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

[ 1  8  5  6 10  0  9  2  3  7  4]


In [64]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Train the model with early stopping
history = tf_model.fit(
    # X_train, y_train,
    X_train_pca, y_train,
    epochs=50,
    # validation_data=(X_val, y_val),
    validation_data=(X_val_pca, y_val),
    callbacks=[early_stopping]
)

Epoch 1/50
2398/2398 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.8124 - loss: 0.7733 - val_accuracy: 0.9901 - val_loss: 0.0515
Epoch 2/50
2398/2398 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9910 - loss: 0.0408 - val_accuracy: 0.9922 - val_loss: 0.0304
Epoch 3/50
2398/2398 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9934 - loss: 0.0256 - val_accuracy: 0.9940 - val_loss: 0.0217
Epoch 4/50
2398/2398 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9948 - loss: 0.0187 - val_accuracy: 0.9953 - val_loss: 0.0169
Epoch 5/50
2398/2398 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9958 - loss: 0.0149 - val_accuracy: 0.9957 - val_loss: 0.0138
Epoch 6/50
2398/2398 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9963 - loss: 0.0127 - val_accuracy: 0.9970 - val_loss: 0.0118
Epoch 7/50
2398/2398 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9975 - loss: 0.0092 - val_accuracy: 0.9970 - val_loss: 0.0102
Epoch 8/50
2398/2398 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9975 - loss: 0.0087 - 

In [65]:
# draw confusion matrix
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

import time 
start = time.time()
y_pred = tf_model(X_test_pca)
end = time.time()
print("Time taken for prediction:", end-start)
y_pred = np.argmax(y_pred, axis=1)

cm = confusion_matrix(y_test, y_pred)
print(cm)

print("\nAccuracy:", np.trace(cm) / np.sum(cm))

Time taken for prediction: 0.017004013061523438
[[1952    2    0    0    0    0    1    0    0    0    0]
 [   3 2622    0    0    0    0    0    0    0    0    0]
 [   0    0 1388    0    0    0    0    0    0    0    0]
 [   0    0    1 1425    0    0    0    0    0    0    0]
 [   0    0    0    0 1443    0    0    0    0    0    0]
 [   0    0    0    0    0 1244    0    0    0    0    0]
 [   0    0    0    0    0    0  997    0    0    0    0]
 [   0    0    0    0    0    0    0 1233    0    0    0]
 [   0    0    0    0    0    0    0    0  862    0    0]
 [   0    0    0    0    0    0    0    0    0 1001    0]
 [   0    0    0    0    0    0    0    0    0    0 2266]]

Accuracy: 0.9995742092457421


In [41]:
# for name, model in my_models:
#     # if name == "Random Forest":
#     if name == "Decision Tree":
#         import pickle
#         with open('../models/d_tree.pkl', 'wb') as f:
#             pickle.dump(model, f)

In [66]:
# Save the model
tf_model.save('../models/tfv4.keras')